In [5]:
import pandas as pd 
import geopandas as gp
from shapely.ops import nearest_points
from shapely.geometry import Point
import numpy as np
from scipy.spatial import cKDTree
path = r'/home/semipro321/Documents/Workspace/Geospatial-Analysis-on-House-Price-Prediction/'
df_house = pd.read_csv(path + 'Inputs/House/house_with_neigh.csv')
df_hospital = pd.read_csv(path + 'Inputs/Hospitals/Hosptials_with_neigh.csv')
gpf_house = gp.GeoDataFrame(df_house, crs="EPSG:4326", geometry=gp.points_from_xy(df_house.long,df_house.lat))
gpf_hosptial = gp.GeoDataFrame(df_hospital, crs="EPSG:4326", geometry=gp.points_from_xy(df_hospital.Long,df_hospital.Lat))


In [7]:
def ckdnearest(gdA, gdB, distcol):
    gdA['geometry'] = gdA.geometry.to_crs('EPSG:5234')
    gdB['geometry'] = gdB.geometry.to_crs('EPSG:5234')

    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    #If you want other features from B
    #gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            #If you want other features from B
            #gdB_nearest,
            pd.Series(dist, name=distcol)
        ], 
        axis=1)

    return gdf

In [8]:
gpf_house = ckdnearest(gpf_house, gpf_hosptial, 'dist_nearest_hospital')

In [11]:
gpf_house['dist_nearest_hospital']

0         130.779836
1          53.767077
2          62.007656
3         310.202984
4         349.138636
            ...     
15229     216.857563
15230     572.164590
15231    3612.816247
15232      54.586746
15233     323.830583
Name: dist_nearest_hospital, Length: 15234, dtype: float64

In [12]:
df_master = pd.read_csv(path + 'Inputs/mastermodel_filledv3.csv')

In [14]:
df_master = pd.merge(df_master, gpf_house['dist_nearest_hospital'], left_index=True, right_index=True)

In [16]:
df_master.to_csv('mastermodel_filledv4.csv', index=False)